# Running Jaccard,Sorensen and Overlap on Multiple GPUs

This is a Multi-GPU notebook that loads data into a dask_cudf dataframe, create a Graph, and then runs Jaccard, Sorensen and Overlap.



| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware        |
|---------------|------------|------------------|-----------------|-----------------------|
| Don Acosta    | 04/21/2023 | created          | 23.06 nightly   |  2xA6000 CUDA 11.7    |


CuGraph's multi-GPU features leverage Dask. RAPIDS has other projects based on Dask such as dask-cudf and dask-cuda. These products will also be used in this example. Check out [RAPIDS.ai](https://rapids.ai/) to learn more about these technologies.

## Basic setup

In [1]:
# Import needed libraries. We recommend using the [cugraph_dev](https://github.com/rapidsai/cugraph/tree/branch-23.02/conda/environments) env through conda
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from cugraph.dask.comms import comms as Comms

import cugraph.dask as dask_cugraph
import cugraph
from cugraph.generators import rmat

import dask_cudf
import time
import urllib.request
import os

### Initialize multi-GPU environment
Before we get started, we need to setup a Dask local cluster of workers to execute our work and a client to coordinate and schedule work for that cluster. As we see below, we can initiate a cluster and client using only 3 lines of code.

 The enable_spilling allows the stored graph to spill to memory on the host if necessary.

In [2]:
def enable_spilling():
    import cudf
    cudf.set_option("spill", True)

In [3]:
enable_spilling()
cluster = LocalCUDACluster()
client = Client(cluster)
client.run(enable_spilling)
Comms.initialize(p2p=True)

2023-04-23 18:47:55,082 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-23 18:47:55,082 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-23 18:47:55,086 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-23 18:47:55,086 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


## Data
This notebook will use RMAT to generate a synthetic dataset.  The size of the dataset will be determined by the number of GPUs present.
This appraoch removes the need to find test data of various sizes.

The notebook will call the cugraph.rmat function and have a dask_cudf DataFrame returned.  The rmat function could simply return a Graph object, which is more memory effecient, but the goal is to show the process starting with a dataframe

| Number of GPUs | Scale | Edge Factor | Est Number of Nodes | Est Number of Edges |
|----------------|-------|-------------|---------------------|---------------------|
| 1              |  24   |     16      |      16,000,000     |      256,000,000    |
| 2              |  25   |     16      |      32,000,000     |      512,000,000    |
| 3              |  25   |     24      |      32,000,000     |      768,000,000    |
| 4              |  26   |     16      |      64,000,000     |    1,024,000,000    |
| 5              |  26   |     20      |      64,000,000     |    1,280,000,000    |
| 6              |  26   |     24      |      64,000,000     |    1,536,000,000    |
| 7              |  26   |     28      |      64,000,000     |    1,792,000,000    |
| 8              |  27   |     16      |     128,000,000     |    2,048,000,000    |

In [4]:
rmat_settings = (
    [24 , 16],
    [25 , 16],
    [25 , 24],
    [26 , 16],
    [26 , 20],
    [26 , 24],
    [26 , 28],
    [27 , 16],
)

In [5]:
import subprocess
def get_gpu_memory_size():
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.total'
            , '--format=csv,nounits,noheader'
        ]
    )    
    return result.decode('utf-8').strip().split('\n')#).strip().split('\n')

gpu_info = get_gpu_memory_size()
number_of_gpus = len(gpu_info)
gpu_memory = int(gpu_info[0])
print(f"the cluster has {number_of_gpus} GPUs where each GPU has {gpu_memory} GB of memory")

the cluster has 2 GPUs where each GPU has 49140 GB of memory


In [6]:
scale, factor = rmat_settings[number_of_gpus - 1]

In [7]:
if gpu_memory < 3600:
    factor = (int)(factor * 0.75)

### Create the RMAT dataset

In [13]:
numedges = (2**scale)* factor
ddf = rmat(
    scale=scale,
    num_edges=numedges,
    a=0.57,
    b=0.19,
    c=0.19,
    seed=42,
    clip_and_flip=False,
    scramble_vertex_ids=True,
    create_using=None,
    mg=True)

## Let's see how many edges were created

In [14]:
len(ddf)

536870912

In [15]:
ddf.head()

,src,dst
0,191098411,67868474
1,103183019,15814086
2,485727031,418544804
3,287302988,244279675
4,248671391,213552775


## Create a Graph
yes, the rmat generator could have returned a Graph, but the goal for for this code to also be used as if data was loaded into cuDF via 
read_csv, read_parquet, or similar data loading process. 

In [17]:
G = cugraph.Graph(directed=False)
G.from_dask_cudf_edgelist(ddf, renumber=False, source='src', destination='dst')

In [18]:
# we are using 1-hop pairs for demonstration
vertex_pairs = ddf.loc[0:1000]

In [20]:
vertex_pairs.head()

,src,dst
0,191098411,67868474
1,103183019,15814086
2,485727031,418544804
3,287302988,244279675
4,248671391,213552775


### Run Multi-GPU jaccard

In [21]:
jdf = dask_cugraph.jaccard(G,vertex_pairs)
jdf.head(20)

,first,second,jaccard_coeff
0,191098411,67868474,0.002960
1,287302988,244279675,0.001012
2,250668373,288160773,0.054598
3,195859914,368672054,0.004346
4,266469247,514244256,0.000000
5,409946724,372136888,0.007824
6,175903387,5307152,0.001650
7,88505003,88354576,0.005608
8,220509840,174092704,0.000832
9,58992883,223720723,0.014142


### Run Multi-GPU Sorensen

In [22]:
sdf = jdf = dask_cugraph.sorensen(G,vertex_pairs)
sdf.head(20)

,first,second,sorensen_coeff
0,191098411,67868474,0.005903
1,287302988,244279675,0.002021
2,250668373,288160773,0.103543
3,195859914,368672054,0.008654
4,266469247,514244256,0.000000
5,409946724,372136888,0.015527
6,175903387,5307152,0.003295
7,88505003,88354576,0.011153
8,220509840,174092704,0.001663
9,58992883,223720723,0.027890


### Run Multi-GPU overlap

In [23]:
odf = jdf = dask_cugraph.overlap(G,vertex_pairs)
odf.head(20)

,first,second,overlap_coeff
0,191098411,67868474,0.030641
1,287302988,244279675,0.085271
2,250668373,288160773,0.103565
3,195859914,368672054,0.018165
4,266469247,514244256,0.000000
5,409946724,372136888,0.030252
6,175903387,5307152,0.049587
7,88505003,88354576,0.423509
8,220509840,174092704,0.025641
9,58992883,223720723,0.144627


### Clean up and Shut down the multi-GPU Environment

In [24]:
del(ddf)
del(G)

Comms.destroy()
client.close()
cluster.close()

___
Copyright (c) 2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___